In [2]:
text = """
 ------------- TRANSFERRING WATER ------------


Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Transferring 21.5 from A1 of Agilent 1 Well Reservoir 290 mL on 1 to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3
        Aspirating 21.5 uL from A1 of Agilent 1 Well Reservoir 290 mL on 1 at 92.86 uL/sec
        Dispensing 21.5 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3 at 92.86 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12

 ------------- TRANSFERRING DNA ------------


Picking up tip from B1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Transferring 10.5 from A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3
        Aspirating 10.5 uL from A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 92.86 uL/sec
        Dispensing 10.5 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3 at 92.86 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
"""

In [3]:
import re
from collections import defaultdict

# Input: Multiline protocol text
# with open("/mnt/data/opentrons_protocol.txt", "r", encoding="utf-8") as file:
#     lines = file.readlines()
text_ = text.replace("\n        ", ";")
lines = text_.strip().split('\n')

# Strip lines, ignore indented lines (children/substeps), and filter empty lines
steps = [line.replace(";", "\n        ").strip() for line in lines if line.strip() and not line.startswith("        ") and not line.startswith("~~") and not "--" in line and not line.endswith(":")]

# Define prepositions to split on
PREPOSITIONS = [' from ', ' to ', ' on ', ' of ', ' into ']

# Structure for collecting parsed results
parsed_steps = []

# Parse each line
for line in steps:
    tokens = [line]
    for prep in PREPOSITIONS:
        new_tokens = []
        for token in tokens:
            new_tokens.extend(token.split(prep))
        tokens = new_tokens
    parsed_steps.append({
        "raw": line,
        "tokens": [t.strip() for t in tokens if t.strip()]
    })

# Group steps into phases based on "Aspirating"
grouped_phases = []
current_phase = []
aspirating_seen = False

last = ""
for step in parsed_steps:
    if (step["raw"].startswith("Aspirating") and not ("Picking up tip" in last) and not ("Moving to" in last) and not ("Transferring" in last)) \
        or ("Picking up tip" in step["raw"]) or ("Moving to" in step["raw"] and not ("Picking up tip" in last)):
        if aspirating_seen:
            grouped_phases.append(current_phase)
            current_phase = []
        aspirating_seen = True
    last = step["raw"]
    current_phase.append(step["raw"])

# Append the last batch
if current_phase:
    grouped_phases.append(current_phase)

# Output the grouped phases
import pandas as pd

grouped_dict = {"Phase {}".format(i + 1): phase for i, phase in enumerate(grouped_phases)}
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in grouped_dict.items() ]))

# Also, extract all distinct actions (first word in each step)
actions = set()
for step in parsed_steps:
    action = step["raw"].split()[0]
    actions.add(action)

sorted_actions = sorted(actions)
sorted_actions


['Dropping', 'Picking', 'Transferring']

In [4]:
df

,Phase 1,Phase 2
0,Picking up tip from A1 of Opentrons OT-2 96 Fi...,Picking up tip from B1 of Opentrons OT-2 96 Fi...
1,Transferring 21.5 from A1 of Agilent 1 Well Re...,Transferring 10.5 from A1 of Bio-Rad 96 Well P...
2,Dropping tip into A1 of Opentrons Fixed Trash ...,Dropping tip into A1 of Opentrons Fixed Trash ...


In [5]:
grouped_dict["Phase 2"]

['Picking up tip from B1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6',
 'Transferring 10.5 from A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3\n        Aspirating 10.5 uL from A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 92.86 uL/sec\n        Dispensing 10.5 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3 at 92.86 uL/sec',
 'Dropping tip into A1 of Opentrons Fixed Trash on 12']

In [6]:
import re
from typing import List, Dict, Optional, Union, Sequence, Literal

def extract_float_after_keyword(text: str, keyword: str) -> Optional[float]:
    match = re.search(fr'{keyword} ([\d.]+)', text)
    return float(match.group(1)) if match else None

def extract_container_from_line(line: str, keyword: str) -> Optional[Dict[str, Union[str, int, float]]]:
    # 匹配 from 模式
    match = re.search(fr'{keyword} [\d.]+ uL .*?from ([A-H]\d+) of (.*?) on (\d+).*?at ([\d.]+) uL/sec', line)
    if not match:
        # 匹配 into 模式
        match = re.search(fr'{keyword} [\d.]+ uL .*?into ([A-H]\d+) of (.*?) on (\d+).*?at ([\d.]+) uL/sec', line)
    if match:
        return {
            "well": match.group(1),
            "labware": match.group(2).strip(),
            "slot": int(match.group(3))
        }
    return None

def is_full_row(wells: List[str]) -> bool:
    """Returns True if all wells in a row (e.g., A1 to A12) are included"""
    if len(wells) < 12:
        return False
    row = wells[0][0]
    indices = sorted([int(w[1:]) for w in wells if w[0] == row])
    return indices == list(range(1, 13))

def build_transfer_liquid_dict_complete(step_lines: List[str]) -> Dict:
    asp_vols = []
    dis_vols = []
    sources = []
    targets = []
    tip_rack_info = None
    asp_flow_rate = None
    dis_flow_rate = None
    blow_out_air_volume = 0.0
    mix_times = 0
    mix_vol = None
    mix_rate = None
    touch_tip = False
    delays = None

    aspirate_index = None
    dispense_index = None
    mixing_indices = []

    # First pass: gather line indices for logic
    for i, line in enumerate(step_lines):
        if line.startswith(" "):  # ignore indented substeps
            continue
        stripped = line.strip()
        if stripped.startswith("Aspirating") and "from" in stripped and aspirate_index is None:
            aspirate_index = i
        elif stripped.startswith("Dispensing") and "into" in stripped and dispense_index is None:
            dispense_index = i
        elif stripped.startswith("Mixing"):
            mixing_indices.append(i)
        elif stripped.startswith("Picking up tip"):
            tip_match = re.search(r'from ([A-H]\d+) of (.*?) on (\d+)', stripped)
            if tip_match:
                tip_rack_info = {
                    "well": tip_match.group(1),
                    "type": tip_match.group(2).strip(),
                    "slot": int(tip_match.group(3))
                }

    # Determine mix_stage
    mix_stage = "none"
    for idx in mixing_indices:
        if aspirate_index is not None and idx < aspirate_index:
            mix_stage = "before" if mix_stage == "none" else "both"
        elif dispense_index is not None and idx > dispense_index:
            mix_stage = "after" if mix_stage == "none" else "both"

    # Second pass: parse actual values
    for i, line in enumerate(step_lines):
        if line.startswith(" "):  # ignore indented substeps
            continue
        stripped = line.strip()

        if stripped.startswith("Aspirating") and "from" in stripped:
            asp_vols = extract_float_after_keyword(stripped, "Aspirating")
            source = extract_container_from_line(stripped, "Aspirating")
            if source:
                sources.append(source)
            asp_flow_rate = extract_float_after_keyword(stripped, "at")
        elif stripped.startswith("Dispensing") and "into" in stripped:
            dis_vols = extract_float_after_keyword(stripped, "Dispensing")
            target = extract_container_from_line(stripped, "Dispensing")
            if target:
                targets.append(target)
            dis_flow_rate = extract_float_after_keyword(stripped, "at")
        
        elif stripped.startswith("Transferring"):
            asp_vols = extract_float_after_keyword(stripped, "Aspirating")
            dis_vols = extract_float_after_keyword(stripped, "Dispensing")
            source = extract_container_from_line(stripped, "Aspirating")
            if source:
                sources.append(source)
            target = extract_container_from_line(stripped, "Dispensing")
            if target:
                targets.append(target)    
                # 新增：分别提取Aspirating和Dispensing的流速
            asp_match = re.search(r"Aspirating.*?at ([\d.]+)", stripped)
            dis_match = re.search(r"Dispensing.*?at ([\d.]+)", stripped)
            asp_flow_rate = float(asp_match.group(1)) if asp_match else None
            dis_flow_rate = float(dis_match.group(1)) if dis_match else None

        elif stripped.startswith("Air gap"):
            blow_out_air_volume = extract_float_after_keyword(stripped, "Aspirating")
        elif stripped.startswith("Mixing"):
            mix_match = re.search(r'Mixing (\d+) times.*?(\d+\.?\d*)', stripped)
            if mix_match:
                mix_times = [int(mix_match.group(1))]
                mix_vol = float(mix_match.group(2))
                mix_rate = extract_float_after_keyword(stripped, "at")
        elif "Touching tip" in stripped:
            touch_tip = True
        elif stripped.startswith("Delaying"):
            delay_match = re.search(r'Delaying for \d+ minutes and ([\d.]+)', stripped)
            if delay_match:
                delays = [int(float(delay_match.group(1)))]

    # Determine 96-well multichannel use
    source_wells = [s['well'] for s in sources]
    target_wells = [t['well'] for t in targets]
    is_96_well = is_full_row(source_wells) and is_full_row(target_wells)

    return {
        "asp_vols": asp_vols,
        "disp_vols": dis_vols,
        "sources": sources,
        "targets": targets,
        "tip_racks": [tip_rack_info] if tip_rack_info else [],
        "use_channels": None,
        "asp_flow_rates": [asp_flow_rate] if asp_flow_rate else None,
        "dis_flow_rates": [dis_flow_rate] if dis_flow_rate else None,
        "offsets": None,
        "touch_tip": touch_tip,
        "liquid_height": None,
        "blow_out_air_volume": [blow_out_air_volume] if blow_out_air_volume else [0.0],
        "spread": "wide",
        "is_96_well": is_96_well,
        "mix_stage": mix_stage,
        "mix_times": mix_times,
        "mix_vol": mix_vol,
        "mix_rate": mix_rate,
        "mix_liquid_height": None,
        "delays": delays
    }


In [7]:
def merge_same_slot_phases(param_dicts: List[Dict]) -> List[Dict]:
    merged = []
    cache = {}
    last_key = None
    last_block = None

    for d in param_dicts:
        if not d["sources"] or not d["targets"]:
            continue
        # 定义用于合并的唯一标识 key，注意引用第一个 blow_out_air_volume 可能为 None
        key = (
            d["sources"][0]["slot"],
            d["targets"][0]["slot"],
            d.get("mix_stage"),
            d.get("is_96_well"),
            d.get("touch_tip"),
            d.get("blow_out_air_volume", [0])[0]
        )

        if (last_key == key and last_block):
            # 合并到上一个 block 中
            for field in ['vols', 'sources', 'targets', 'tip_racks', 'flow_rates', 'blow_out_air_volume', 'delays']:
                if field in d:
                    if not isinstance(last_block[field], list):
                        last_block[field] = [last_block[field]]
                    if isinstance(d[field], list):
                        last_block[field].extend(d[field])
                    else:
                        last_block[field].append(d[field])
        else:
            # 启动新的 merge block
            merged.append(d)
            last_block = d
            last_key = key

    return merged

In [8]:
outputs = [build_transfer_liquid_dict_complete(steps) for phase, steps in grouped_dict.items()]
outputs

[{'asp_vols': 21.5,
  'disp_vols': 21.5,
  'sources': [{'well': 'A1',
    'labware': 'Agilent 1 Well Reservoir 290 mL',
    'slot': 1}],
  'targets': [{'well': 'A1',
    'labware': 'Bio-Rad 96 Well Plate 200 µL PCR',
    'slot': 3}],
  'tip_racks': [{'well': 'A1',
    'type': 'Opentrons OT-2 96 Filter Tip Rack 200 µL',
    'slot': 6}],
  'use_channels': None,
  'asp_flow_rates': [92.86],
  'dis_flow_rates': [92.86],
  'offsets': None,
  'touch_tip': False,
  'liquid_height': None,
  'blow_out_air_volume': [0.0],
  'spread': 'wide',
  'is_96_well': False,
  'mix_stage': 'none',
  'mix_times': 0,
  'mix_vol': None,
  'mix_rate': None,
  'mix_liquid_height': None,
  'delays': None},
 {'asp_vols': 10.5,
  'disp_vols': 10.5,
  'sources': [{'well': 'A1',
    'labware': 'Bio-Rad 96 Well Plate 200 µL PCR',
    'slot': 2}],
  'targets': [{'well': 'A1',
    'labware': 'Bio-Rad 96 Well Plate 200 µL PCR',
    'slot': 3}],
  'tip_racks': [{'well': 'B1',
    'type': 'Opentrons OT-2 96 Filter Tip Rac

In [ ]:
o = merge_same_slot_phases(outputs)
ddf = pd.DataFrame({"Phase {}".format(i + 1): phase for i, phase in enumerate(o)})
ddf

,Phase 1,Phase 2
asp_vols,21.5,10.5
disp_vols,21.5,10.5
sources,"[{'well': 'A1', 'labware': 'Agilent 1 Well Res...","[{'well': 'A1', 'labware': 'Bio-Rad 96 Well Pl..."
targets,"[{'well': 'A1', 'labware': 'Bio-Rad 96 Well Pl...","[{'well': 'A1', 'labware': 'Bio-Rad 96 Well Pl..."
tip_racks,"[{'well': 'A1', 'type': 'Opentrons OT-2 96 Fil...","[{'well': 'B1', 'type': 'Opentrons OT-2 96 Fil..."
use_channels,None,None
asp_flow_rates,[92.86],[92.86]
dis_flow_rates,[92.86],[92.86]
offsets,None,None
touch_tip,False,False


In [11]:
text2 = """
Setting Temperature Module temperature to 6.0 °C (rounded off to nearest integer)
Adding Proteinase K to each well:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H3 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H4 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H5 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into B11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at B11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into C11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at C11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from D6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into D11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at D11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from E6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into E11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at E11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from F6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into F11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at F11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from G6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into G11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at G11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from H6 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into H11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at H11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Adding viral buffer + beads to samples:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A1 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A2 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 160.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 160.0 uL from A3 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 200.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Mixing samples+buffer+beads:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Engaging Magnetic Module
Incubating on magdeck for 5 minutes
Delaying for 5 minutes and 0.0 seconds
Removing supernatant:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 160.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 190.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Disengaging Magnetic Module
Wash Step 1 Wash Buffer 1 - Adding to samples:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A4 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A5 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Engaging Magnetic Module
Incubating on MagDeck for 3 minutes.
Delaying for 3 minutes and 0.0 seconds
Removing supernatant from Wash 1 Wash Buffer 1:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 6
Disengaging Magnetic Module
Wash Step 2 Wash Buffer 2 - Adding to samples:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A6 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A7 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Engaging Magnetic Module
Incubating on MagDeck for 3 minutes.
Delaying for 3 minutes and 0.0 seconds
Removing supernatant from Wash 2 Wash Buffer 2:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Disengaging Magnetic Module
Wash Step 3 Ethanol 1 - Adding to samples:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A8 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A9 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Engaging Magnetic Module
Incubating on MagDeck for 3 minutes.
Delaying for 3 minutes and 0.0 seconds
Removing supernatant from Wash 3 Ethanol 1:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 9
Disengaging Magnetic Module
Wash Step 4 Ethanol 2 - Adding to samples:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A10 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 165.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 10.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 165.0 uL from A11 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 185.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 20.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dispensing 20.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Returning tip
        Dropping tip into A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Engaging Magnetic Module
Incubating on MagDeck for 3 minutes.
Delaying for 3 minutes and 0.0 seconds
Removing supernatant from Wash 4 Ethanol 2:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Aspirating 10.0 uL from A1 of Liquid Waste on 11 at 20.0 uL/sec
Aspirating 135.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
Dispensing 165.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Returning tip
        Dropping tip into A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Disengaging Magnetic Module
Allowing beads to air dry for 2 minutes.
Delaying for 2 minutes and 0.0 seconds
Removing any excess ethanol from wells:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 7
Transferring 180.0 from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 to A1 of Liquid Waste on 11
        Aspirating 180.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 20.0 uL/sec
        Dispensing 180.0 uL into A1 of Liquid Waste on 11 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Allowing beads to air dry for 10 minutes.
Delaying for 10 minutes and 0.0 seconds
Disengaging Magnetic Module
Adding NF-Water to wells for elution:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A3 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A4 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A5 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A6 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 20.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Aspirating 50.0 uL from A12 of Trough with Reagents on 2 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 40.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 40.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 70.0 uL into A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Blowing out at A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Incubating at room temp for 2 minutes.
Delaying for 2 minutes and 0.0 seconds
Engaging Magnetic Module
Incubating on MagDeck for 4 minutes.
Delaying for 4 minutes and 0.0 seconds
Transferring elution to final plate:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A1 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A2 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A9 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A5 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A3 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A10 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A7 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A4 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A11 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A9 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A5 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A12 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 10
Aspirating 50.0 uL from A11 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 10.0 uL/sec
Dispensing 50.0 uL into A6 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module GEN1 on 3 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Disengaging Magnetic Module
Congratulations! The protocol is now complete!
"""

In [42]:
text2 = """
Setting Temperature Module temperature to 6.0 °C (rounded off to nearest integer)
Adding Proteinase K to each well:
Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5
Aspirating 4.0 uL from D2 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec
Dispensing 4.0 uL into A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec
Blowing out at A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4
Dropping tip into A1 of Opentrons Fixed Trash on 12
Mixing samples+buffer+beads:
Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Aspirating 180.0 uL from A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A8 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1
Aspirating 20.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Aspirating 180.0 uL from A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 50.0 uL/sec
Dispensing 180.0 uL into A3 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 150.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
"""

In [51]:
import re
from collections import defaultdict

# Input: Multiline protocol text
# with open("/mnt/data/opentrons_protocol.txt", "r", encoding="utf-8") as file:
#     lines = file.readlines()
text_ = text2.replace("\n        ", ";")
lines = text_.strip().split('\n')

# Strip lines, ignore indented lines (children/substeps), and filter empty lines
steps = [line.replace(";", "\n        ").strip() for line in lines if line.strip() and not line.startswith("        ") and not line.startswith("~~") and not "--" in line and not line.endswith(":")]
steps

['Setting Temperature Module temperature to 6.0 °C (rounded off to nearest integer)',
 'Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5',
 'Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec',
 'Dispensing 4.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec',
 'Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4',
 'Dropping tip into A1 of Opentrons Fixed Trash on 12',
 'Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5',
 'Aspirating 4.0 uL from D2 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec',
 'Dispensing 4.0 uL into A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec',
 'Blowing out at A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4',
 'Dropping tip into A1 of Opentrons Fixed Trash on 12',
 'Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1',
 'Aspirating 20.0 uL from A1 of NEST 96 Deep Well Pl

In [52]:

# Define prepositions to split on
PREPOSITIONS = [' from ', ' to ', ' on ', ' of ', ' into ']

# Structure for collecting parsed results
parsed_steps = []

# Parse each line
for line in steps:
    tokens = [line]
    for prep in PREPOSITIONS:
        new_tokens = []
        for token in tokens:
            new_tokens.extend(token.split(prep))
        tokens = new_tokens
    parsed_steps.append({
        "raw": line,
        "tokens": [t.strip() for t in tokens if t.strip()]
    })

# Group steps into phases based on "Aspirating"
grouped_phases = []
current_phase = []
aspirating_seen = False

last = ""
for step in parsed_steps:
    if (step["raw"].startswith("Aspirating") and not ("Picking up tip" in last) and not ("Moving to" in last) and not ("Transferring" in last)) \
        or ("Picking up tip" in step["raw"]) or ("Moving to" in step["raw"] and not ("Picking up tip" in last)):
        if aspirating_seen:
            grouped_phases.append(current_phase)
            current_phase = []
        aspirating_seen = True
    last = step["raw"]
    current_phase.append(step["raw"])

# Append the last batch
if current_phase:
    grouped_phases.append(current_phase)

In [53]:
grouped_phases

[['Setting Temperature Module temperature to 6.0 °C (rounded off to nearest integer)',
  'Picking up tip from A1 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5',
  'Aspirating 4.0 uL from D1 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec',
  'Dispensing 4.0 uL into A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec',
  'Blowing out at A1 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4',
  'Dropping tip into A1 of Opentrons Fixed Trash on 12'],
 ['Picking up tip from A2 of Opentrons OT-2 96 Filter Tip Rack 20 µL on 5',
  'Aspirating 4.0 uL from D2 of Opentrons 24 TubeRack on 8 at 3.78 uL/sec',
  'Dispensing 4.0 uL into A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4 at 3.78 uL/sec',
  'Blowing out at A2 of NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1 on 4',
  'Dropping tip into A1 of Opentrons Fixed Trash on 12'],
 ['Picking up tip from A7 of Opentrons OT-2 96 Filter Tip Rack 200 µL on 1',
  'Aspirating 20.0 uL from A1 of NEST 

In [54]:
# Output the grouped phases
import pandas as pd

grouped_dict = {"Phase {}".format(i + 1): phase for i, phase in enumerate(grouped_phases)}
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in grouped_dict.items() ]))

# Also, extract all distinct actions (first word in each step)
actions = set()
for step in parsed_steps:
    action = step["raw"].split()[0]
    actions.add(action)

sorted_actions = sorted(actions)
sorted_actions

['Aspirating', 'Blowing', 'Dispensing', 'Dropping', 'Picking', 'Setting']

In [55]:
df

,Phase 1,Phase 2,Phase 3,Phase 4,Phase 5,Phase 6,Phase 7
0,Setting Temperature Module temperature to 6.0 ...,Picking up tip from A2 of Opentrons OT-2 96 Fi...,Picking up tip from A7 of Opentrons OT-2 96 Fi...,Aspirating 180.0 uL from A1 of NEST 96 Deep We...,Aspirating 180.0 uL from A1 of NEST 96 Deep We...,Picking up tip from A8 of Opentrons OT-2 96 Fi...,Aspirating 180.0 uL from A3 of NEST 96 Deep We...
1,Picking up tip from A1 of Opentrons OT-2 96 Fi...,Aspirating 4.0 uL from D2 of Opentrons 24 Tube...,Aspirating 20.0 uL from A1 of NEST 96 Deep Wel...,Dispensing 180.0 uL into A1 of NEST 96 Deep We...,Dispensing 180.0 uL into A1 of NEST 96 Deep We...,Aspirating 20.0 uL from A3 of NEST 96 Deep Wel...,Dispensing 180.0 uL into A3 of NEST 96 Deep We...
2,Aspirating 4.0 uL from D1 of Opentrons 24 Tube...,Dispensing 4.0 uL into A2 of NEST 96 Deep Well...,NaN,NaN,Dropping tip into A1 of Opentrons Fixed Trash ...,NaN,Dropping tip into A1 of Opentrons Fixed Trash ...
3,Dispensing 4.0 uL into A1 of NEST 96 Deep Well...,Blowing out at A2 of NEST 96 Deep Well Plate 2...,NaN,NaN,NaN,NaN,NaN
4,Blowing out at A1 of NEST 96 Deep Well Plate 2...,Dropping tip into A1 of Opentrons Fixed Trash ...,NaN,NaN,NaN,NaN,NaN
5,Dropping tip into A1 of Opentrons Fixed Trash ...,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
import re
from typing import List, Dict, Optional, Union, Sequence, Literal

def extract_float_after_keyword(text: str, keyword: str) -> Optional[float]:
    match = re.search(fr'{keyword} ([\d.]+)', text)
    return float(match.group(1)) if match else None

def extract_container_from_line(line: str, keyword: str) -> Optional[Dict[str, Union[str, int, float]]]:
    # 匹配 from 模式
    match = re.search(fr'{keyword} [\d.]+ uL .*?from ([A-H]\d+) of (.*?) on (\d+).*?at ([\d.]+) uL/sec', line)
    if not match:
        # 匹配 into 模式
        match = re.search(fr'{keyword} [\d.]+ uL .*?into ([A-H]\d+) of (.*?) on (\d+).*?at ([\d.]+) uL/sec', line)
    if match:
        return {
            "well": match.group(1),
            "labware": match.group(2).strip(),
            "slot": int(match.group(3))
        }
    return None

def is_full_row(wells: List[str]) -> bool:
    """Returns True if all wells in a row (e.g., A1 to A12) are included"""
    if len(wells) < 12:
        return False
    row = wells[0][0]
    indices = sorted([int(w[1:]) for w in wells if w[0] == row])
    return indices == list(range(1, 13))

def build_transfer_liquid_dict_complete(step_lines: List[str]) -> Dict:
    asp_vols = []
    dis_vols = []
    sources = []
    targets = []
    tip_rack_info = None
    asp_flow_rate = None
    dis_flow_rate = None
    blow_out_air_volume = 0.0
    mix_times = 0
    mix_vol = None
    mix_rate = None
    touch_tip = False
    delays = None

    aspirate_index = None
    dispense_index = None
    mixing_indices = []

    # First pass: gather line indices for logic
    for i, line in enumerate(step_lines):
        if line.startswith(" "):  # ignore indented substeps
            continue
        stripped = line.strip()
        if stripped.startswith("Aspirating") and "from" in stripped and aspirate_index is None:
            aspirate_index = i
        elif stripped.startswith("Dispensing") and "into" in stripped and dispense_index is None:
            dispense_index = i
        elif stripped.startswith("Mixing"):
            mixing_indices.append(i)
        elif stripped.startswith("Picking up tip"):
            tip_match = re.search(r'from ([A-H]\d+) of (.*?) on (\d+)', stripped)
            if tip_match:
                tip_rack_info = {
                    "well": tip_match.group(1),
                    "type": tip_match.group(2).strip(),
                    "slot": int(tip_match.group(3))
                }

    # Determine mix_stage
    mix_stage = "none"
    for idx in mixing_indices:
        if aspirate_index is not None and idx < aspirate_index:
            mix_stage = "before" if mix_stage == "none" else "both"
        elif dispense_index is not None and idx > dispense_index:
            mix_stage = "after" if mix_stage == "none" else "both"

    # Second pass: parse actual values
    for i, line in enumerate(step_lines):
        if line.startswith(" "):  # ignore indented substeps
            continue
        stripped = line.strip()

        if stripped.startswith("Aspirating") and "from" in stripped:
            asp_vols = extract_float_after_keyword(stripped, "Aspirating")
            source = extract_container_from_line(stripped, "Aspirating")
            if source:
                sources.append(source)
            asp_flow_rate = extract_float_after_keyword(stripped, "at")
        elif stripped.startswith("Dispensing") and "into" in stripped:
            dis_vols = extract_float_after_keyword(stripped, "Dispensing")
            target = extract_container_from_line(stripped, "Dispensing")
            if target:
                targets.append(target)
            dis_flow_rate = extract_float_after_keyword(stripped, "at")
        
        elif stripped.startswith("Transferring"):
            asp_vols = extract_float_after_keyword(stripped, "Aspirating")
            dis_vols = extract_float_after_keyword(stripped, "Dispensing")
            source = extract_container_from_line(stripped, "Aspirating")
            if source:
                sources.append(source)
            target = extract_container_from_line(stripped, "Dispensing")
            if target:
                targets.append(target)    
                # 新增：分别提取Aspirating和Dispensing的流速
            asp_match = re.search(r"Aspirating.*?at ([\d.]+)", stripped)
            dis_match = re.search(r"Dispensing.*?at ([\d.]+)", stripped)
            asp_flow_rate = float(asp_match.group(1)) if asp_match else None
            dis_flow_rate = float(dis_match.group(1)) if dis_match else None

        elif stripped.startswith("Air gap"):
            blow_out_air_volume = extract_float_after_keyword(stripped, "Aspirating")
        elif stripped.startswith("Mixing"):
            mix_match = re.search(r'Mixing (\d+) times.*?(\d+\.?\d*)', stripped)
            if mix_match:
                mix_times = [int(mix_match.group(1))]
                mix_vol = float(mix_match.group(2))
                mix_rate = extract_float_after_keyword(stripped, "at")
        elif "Touching tip" in stripped:
            touch_tip = True
        elif stripped.startswith("Delaying"):
            delay_match = re.search(r'Delaying for \d+ minutes and ([\d.]+)', stripped)
            if delay_match:
                delays = [int(float(delay_match.group(1)))]

    # Determine 96-well multichannel use
    source_wells = [s['well'] for s in sources]
    target_wells = [t['well'] for t in targets]
    is_96_well = is_full_row(source_wells) and is_full_row(target_wells)

    return {
        "asp_vols": asp_vols,
        "disp_vols": dis_vols,
        "sources": sources,
        "targets": targets,
        "tip_racks": [tip_rack_info] if tip_rack_info else [],
        "use_channels": None,
        "asp_flow_rates": [asp_flow_rate] if asp_flow_rate else None,
        "dis_flow_rates": [dis_flow_rate] if dis_flow_rate else None,
        "offsets": None,
        "touch_tip": touch_tip,
        "liquid_height": None,
        "blow_out_air_volume": [blow_out_air_volume] if blow_out_air_volume else [0.0],
        "spread": "wide",
        "is_96_well": is_96_well,
        "mix_stage": mix_stage,
        "mix_times": mix_times,
        "mix_vol": mix_vol,
        "mix_rate": mix_rate,
        "mix_liquid_height": None,
        "delays": delays
    }


In [57]:
def merge_same_slot_phases(param_dicts: List[Dict]) -> List[Dict]:
    merged = []
    cache = {}
    last_key = None
    last_block = None

    for d in param_dicts:
        if not d["sources"] or not d["targets"]:
            continue
        # 定义用于合并的唯一标识 key，注意引用第一个 blow_out_air_volume 可能为 None
        key = (
            d["sources"][0]["slot"],
            d["targets"][0]["slot"],
            d.get("mix_stage"),
            d.get("is_96_well"),
            d.get("touch_tip"),
            d.get("blow_out_air_volume", [0])[0]
        )

        if (last_key == key and last_block):
            # 合并到上一个 block 中
            for field in ['vols', 'sources', 'targets', 'tip_racks', 'flow_rates', 'blow_out_air_volume', 'delays']:
                if field in d:
                    if not isinstance(last_block[field], list):
                        last_block[field] = [last_block[field]]
                    if isinstance(d[field], list):
                        last_block[field].extend(d[field])
                    else:
                        last_block[field].append(d[field])
        else:
            # 启动新的 merge block
            merged.append(d)
            last_block = d
            last_key = key

    return merged

In [58]:
outputs = [build_transfer_liquid_dict_complete(steps) for phase, steps in grouped_dict.items()]
outputs

[{'asp_vols': 4.0,
  'disp_vols': 4.0,
  'sources': [{'well': 'D1', 'labware': 'Opentrons 24 TubeRack', 'slot': 8}],
  'targets': [{'well': 'A1',
    'labware': 'NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1',
    'slot': 4}],
  'tip_racks': [{'well': 'A1',
    'type': 'Opentrons OT-2 96 Filter Tip Rack 20 µL',
    'slot': 5}],
  'use_channels': None,
  'asp_flow_rates': [3.78],
  'dis_flow_rates': [3.78],
  'offsets': None,
  'touch_tip': False,
  'liquid_height': None,
  'blow_out_air_volume': [0.0],
  'spread': 'wide',
  'is_96_well': False,
  'mix_stage': 'none',
  'mix_times': 0,
  'mix_vol': None,
  'mix_rate': None,
  'mix_liquid_height': None,
  'delays': None},
 {'asp_vols': 4.0,
  'disp_vols': 4.0,
  'sources': [{'well': 'D2', 'labware': 'Opentrons 24 TubeRack', 'slot': 8}],
  'targets': [{'well': 'A2',
    'labware': 'NEST 96 Deep Well Plate 2mL on Magnetic Module GEN1',
    'slot': 4}],
  'tip_racks': [{'well': 'A2',
    'type': 'Opentrons OT-2 96 Filter Tip Rack 20 µ

In [59]:
o = merge_same_slot_phases(outputs)
ddf = pd.DataFrame({"Phase {}".format(i + 1): phase for i, phase in enumerate(o)})
ddf

,Phase 1,Phase 2
asp_vols,4.0,180.0
disp_vols,4.0,180.0
sources,"[{'well': 'D1', 'labware': 'Opentrons 24 TubeR...","[{'well': 'A1', 'labware': 'NEST 96 Deep Well ..."
targets,"[{'well': 'A1', 'labware': 'NEST 96 Deep Well ...","[{'well': 'A1', 'labware': 'NEST 96 Deep Well ..."
tip_racks,"[{'well': 'A1', 'type': 'Opentrons OT-2 96 Fil...",[]
use_channels,None,None
asp_flow_rates,[3.78],[50.0]
dis_flow_rates,[3.78],[150.0]
offsets,None,None
touch_tip,False,False
